In [2]:
import pandas as pd
import openpyxl
from pathlib import Path

In [14]:
#This function is to extract data from xslx file that we have in data folder
def extract_data_from_xslx(sheet_name=None):
    df = pd.read_excel(sheet_name, engine="openpyxl", header=0, dtype=str)
    return df

In [6]:
#This function is to clean column names of a dataframe
def clean_column_names(df):
    df.columns = df.columns.str.strip().str.lower().str.replace(" ", "_").str.replace("(", "").str.replace(")", "")
    return df

In [7]:
#Function to delete specific columns from a dataframe
def delete_columns(df, columns_to_delete):
    df = df.drop(columns=columns_to_delete)
    return df

In [8]:
def filter_rows_by_value(df, column_name, value, comparison_type='equals'):
    if comparison_type == 'equals':
        filtered_df = df[df[column_name] == value]
    elif comparison_type == 'not_equals':
        filtered_df = df[df[column_name] != value]
    elif comparison_type == 'greater_than':
        filtered_df = df[df[column_name] > value]
    elif comparison_type == 'less_than':
        filtered_df = df[df[column_name] < value]
    else:
        raise ValueError("Invalid comparison_type. Use 'equals', 'not_equals', 'greater_than', or 'less_than'.")
    return filtered_df

In [18]:
#Main function to call the other functions

df_abastecimientos = extract_data_from_xslx("./Data/abastecimientos_sap.xlsx")
df_actividades = extract_data_from_xslx("./Data/actividades.xlsx")
df_insumos = extract_data_from_xslx("./Data/detalle_apuntamiento_insumos.xlsx")
df_rep_maquinaria = extract_data_from_xslx("./Data/rep_maquinaria.xlsx")


First Excel

In [9]:
df_abastecimientos = extract_data_from_xslx("./Data/abastecimientos_sap.xlsx")
clean_column_names(df_abastecimientos)


#Con este filtrado, se eliminan tambien los vacios de la columna de equipo
df_abastecimientos = filter_rows_by_value(df_abastecimientos, 'clase_de_movimiento', 261)

df_abastecimientos = filter_rows_by_value(df_abastecimientos, 'centro_de_coste', 2000000, 'less_than')
    
#Eliminar columnas innecesarias para el dataframe de abastecimientos
innecesary_columns = ['material', 'texto_breve_de_material', 'almacén', 'clase_de_movimiento', 'posición_doc.mat.',
                          'nº_reserva', 'centro_de_coste', 'un.medida_de_entrada']
for column in innecesary_columns:
    if column in df_abastecimientos.columns:
        df_abastecimientos = delete_columns(df_abastecimientos, [column])

df_abastecimientos.rename(columns={'fe.contabilización': 'fecha'}, inplace=True)
df_abastecimientos.rename(columns={'orden': 'equipo'}, inplace=True)
df_abastecimientos.rename(columns={'ctd.en_um_entrada': 'galones'}, inplace=True)

#eliminar los tres primeros caracteres del equipo
df_abastecimientos['equipo'] = df_abastecimientos['equipo'].astype(str).str[3:]

display(df_abastecimientos.head())

,equipo,fecha,galones
0,TRC1-7548,2025-09-09,-157.233
1,CTA1-8552,2025-09-09,-15.535
2,TRC1-7249,2025-09-09,-19.917
3,TRC1-7246,2025-09-09,-15.366
4,TRC1-7406,2025-09-09,-30.280


Second Excel

In [18]:
df_actividades = extract_data_from_xslx("./Data/actividades.xlsx")
clean_column_names(df_actividades)

    
#Eliminar columnas innecesarias para el dataframe de abastecimientos
innecesary_columns = ["tenencia", "id", "nom._empresa", "parada", "mov_parada", "fch_cort_siem"]
for column in innecesary_columns:
    if column in df_actividades.columns:
        df_actividades = delete_columns(df_actividades, [column])

#crear nueva columna para un id unico que sea la concatenacion fazenda lote y talhao y que quede de primera columna
df_actividades.insert(0, 'id', '')
df_actividades['id'] = df_actividades['fazenda'].astype(str) + '_' + df_actividades['lote'].astype(str) + '_' + df_actividades['talhao'].astype(str)

display(df_actividades.head())

,id,empresa,fazenda,lote,talhao,ccusto,cencos,oper,actividad,data,quantidade,a_pag_uni,valor_total,unidade,um_prod,doc_erp1,os,func,qtd_prod,area_real_suerte
0,014_0005_0000,99,014,0005,0000,1010042,Fertilización,18601,Cargue manual de fertilizantes,2025-08-01 00:00:00,7,9393.836,65756.852,H,H,NaN,821891,088194,7,30.12
1,001_0053_A000,99,001,0053,A000,1010042,Fertilización,18601,Cargue manual de fertilizantes,2025-08-01 00:00:00,7,8430.383,59012.681,H,H,NaN,821713,087821,7,11.58
2,014_0005_0000,99,014,0005,0000,1010042,Fertilización,18601,Cargue manual de fertilizantes,2025-08-01 00:00:00,7,8430.383,59012.681,H,H,NaN,821891,088449,7,30.12
3,014_0005_0000,99,014,0005,0000,1010042,Fertilización,18601,Cargue manual de fertilizantes,2025-08-01 00:00:00,7,8430.383,59012.681,H,H,NaN,821891,000607,7,30.12
4,014_0005_0000,99,014,0005,0000,1010042,Fertilización,18601,Cargue manual de fertilizantes,2025-08-01 00:00:00,7,8430.383,59012.681,H,H,NaN,821891,000305,7,30.12
